#### INFO


***INITIAL SETUP***

1. *Runtime ->  Change Runtime Type -> Python 3 + GPU*
2. *Fill in `GDRIVE_DIR` in [code #5] (name of the folder in gdrive that contains your code and data)*

***RUNNING THE SCRIPTS***
1. *Runtime -> Run All* before the first run. 
2. Pay attention to the authentification. Also, sometimes so you may need to clean the drive cache ([code #5])
4. Training - when re-running the training, you **do not** have to run any of the preceding scripts. However, Clearing google drive cache might help you.

***ABOUT***

- *[code #1] Install dependencies for google authentification*
- *[code #2] Allow collaboratory to read from & write into your google drive"*
- *[code #3] Download tensorboard accesories*
- *[code #4] Setup paths for tensorboard and your script. *
- *[code #5] Initialize the functions for tensorflow and runner function*
- *[code #6] Mounts the google drive to current FS as ./drive.*
- *[code #7] VM reset script - Kills all processes. Sometimes helps, once the GPU tensorflow stops working, drive fails etc. **! RESETS AUTH !** *
- *[code #8] Sample call to run function that actually runs tensorboard and your script.*


***FINAL NOTES***

Most of this script was copy-pasted from https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

The Tensorflow part is taken from https://github.com/mixuala/colab_utils.git + Special thanks to Štěpán Hojdar.

Feel free to improve this script - just send me suggestions / bug reports / improved scripts and I will upload it here.

Created by Vladislav Vancák & Petr Houška.

#### SETUP

In [1]:
# DRIVE & AUTH DEPENDENCIES
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) 

In [2]:
# TENSORBOARD DEPENDENCIES
!git clone https://github.com/mixuala/colab_utils.git

Cloning into 'colab_utils'...
remote: Counting objects: 219, done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 219 (delta 16), reused 43 (delta 13), pack-reused 171
Receiving objects: 100% (219/219), 60.62 KiB | 6.06 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [15]:
# GOOGLE AUTH
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

ROOT = None

··········


#### CONFIG

In [0]:
# SCRIPT PROPERTIES !!! CHANGE GOOGLE DRIVE FOLDER HERE e.g. GDRIVE_DIR="MFF/DL/mnist" !!!
import os

GDRIVE_DIR="DL_3drecognition"

if ROOT is not None:
  os.chdir(ROOT)

ROOT = %pwd
SCRIPT_PATH = os.path.join(ROOT, 'drive', GDRIVE_DIR)
LOG_DIR = os.path.join(SCRIPT_PATH, 'logs')

In [0]:
# RUNNER FUNCTION
import tensorflow as tf
def run_script(name, parameters):
  SCRIPT_FILE= name
  SCRIPT_ARGS= parameters

  os.chdir(SCRIPT_PATH)

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print('[WARN]: GPU device not found. Running in CPU mode')
  else:
    print('[INFO]: Found GPU at: {}'.format(device_name))  
  print('<<= <=> =>>')
  
  %run $SCRIPT_FILE $SCRIPT_ARGS

# TENSORBOARD
def tensorboard():
  
  import os
  import sys
  import colab_utils.tboard
  
  os.chdir(ROOT)
  colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR)

# GOOGLE DRIVE MOUNT / CLEAR CACHE
def google_drive():
  os.chdir(ROOT)
  
  if os.path.exists('drive'):
    !google-drive-ocamlfuse -cc

  else:
    !mkdir -p drive
    !google-drive-ocamlfuse drive

# RUN ALL
def run(name, parameters):
  google_drive()
  print('<<= <=> =>>')

  tensorboard()
  print('<<= <=> =>>')
  
  run_script(name, parameters)

#### UTILS

In [0]:
# VM RESET !!! RESETS GOOGLE AUTH !!
# !kill -9 -1 

#### RUN

In [10]:
run(
    "3d_recognition.py",
    "--epochs 20"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= https://b6ee92ce.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.7650
1|acc:0.8950
2|acc:0.9150
3|acc:0.9275
4|acc:0.9325
5|acc:0.9450
6|acc:0.9475
7|acc:0.9575
8|acc:0.9625
9|acc:0.9575
10|acc:0.9650
11|acc:0.9600
12|acc:0.9600
13|acc:0.9700
14|acc:0.9600
15|acc:0.9575
16|acc:0.9625
17|acc:0.9550
18|acc:0.9575
19|acc:0.9600


In [15]:
run(
    "3d_recognition.py",
    "--epochs 20 --cnn C-32-3-1-same,C-32-3-1-same,M-3-2,F"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= https://b6ee92ce.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.9200
1|acc:0.9475
2|acc:0.9525
3|acc:0.9525
4|acc:0.9450
5|acc:0.9475
6|acc:0.9475
7|acc:0.9575
8|acc:0.9500
9|acc:0.9500
10|acc:0.9550
11|acc:0.9525
12|acc:0.9525
13|acc:0.9600
14|acc:0.9550
15|acc:0.9550
16|acc:0.9550
17|acc:0.9575
18|acc:0.9525
19|acc:0.9575


In [16]:
run(
    "3d_recognition.py",
    "--epochs 20 --cnn C-32-3-1-same,C-32-3-1-same,M-3-2,C-64-3-1-same,C-64-3-1-same,M-3-2,F"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= https://b6ee92ce.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.8450
1|acc:0.9175
2|acc:0.9425
3|acc:0.9425
4|acc:0.9500
5|acc:0.9650
6|acc:0.9575
7|acc:0.9650
8|acc:0.9675
9|acc:0.9625
10|acc:0.9650
11|acc:0.9625
12|acc:0.9650
13|acc:0.9650
14|acc:0.9600
15|acc:0.9625
16|acc:0.9650
17|acc:0.9625
18|acc:0.9600
19|acc:0.9625


In [17]:
run(
    "3d_recognition.py",
    "--epochs 30 --cnn C-64-3-1-same,C-64-3-1-same,M-3-2,C-128-3-1-same,C-128-3-1-same,M-3-2,C-128-3-1-same,C-128-3-1-same,F"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://b6ee92ce.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.2200
1|acc:0.6175
2|acc:0.8125
3|acc:0.8950
4|acc:0.9275
5|acc:0.9325
6|acc:0.9175
7|acc:0.9325
8|acc:0.9525
9|acc:0.9575
10|acc:0.9525
11|acc:0.9550
12|acc:0.9550
13|acc:0.9550
14|acc:0.9525
15|acc:0.9575
16|acc:0.9600
17|acc:0.9475
18|acc:0.9525
19|acc:0.9575
20|acc:0.9550
21|acc:0.9525
22|acc:0.9575
23|acc:0.9550
24|acc:0.9550
25|acc:0.9525
26|acc:0.9575
27|acc:0.9525
28|acc:0.9550
29|acc:0.9575


In [18]:
run(
    "3d_recognition.py",
    "--epochs 20 --cnn CB-64-3-1-same,CB-64-3-1-same,M-3-2,CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-128-3-1-same,CB-128-3-1-same,F"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://b6ee92ce.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.1775
1|acc:0.6300
2|acc:0.1425
3|acc:0.2750
4|acc:0.2475
5|acc:0.2550
6|acc:0.2525
7|acc:0.3500
8|acc:0.3725
9|acc:0.5825
10|acc:0.5925
11|acc:0.6975
12|acc:0.7575
13|acc:0.7850
14|acc:0.7275
15|acc:0.8525
16|acc:0.8425
17|acc:0.8725
18|acc:0.9075
19|acc:0.9300


In [10]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 20 --cnn C-32-3-1-same,C-32-3-1-same,M-3-2,C-32-3-1-same,C-32-3-1-same,M-3-2,F --modelnet_dim 32"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=False
tensorboard url= http://bd50c7c5.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.9050
1|acc:0.9525
2|acc:0.9375
3|acc:0.9500
4|acc:0.9525
5|acc:0.9650
6|acc:0.9450
7|acc:0.9550
8|acc:0.9575
9|acc:0.9600
10|acc:0.9625
11|acc:0.9600
12|acc:0.9675
13|acc:0.9600
14|acc:0.9625
15|acc:0.9625
16|acc:0.9650
17|acc:0.9625
18|acc:0.9650
19|acc:0.9650


In [12]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 30 --cnn C-32-3-1-same,C-32-3-1-same,M-3-2,C-32-3-1-same,C-32-3-1-same,M-3-2,F --modelnet_dim 32 --train_split 0.8"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://bd50c7c5.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.8999
1|acc:0.9337
2|acc:0.9374
3|acc:0.9437
4|acc:0.9537
5|acc:0.9499
6|acc:0.9574
7|acc:0.9512
8|acc:0.9549
9|acc:0.9549
10|acc:0.9537
11|acc:0.9524
12|acc:0.9574
13|acc:0.9599
14|acc:0.9562
15|acc:0.9562
16|acc:0.9637
17|acc:0.9574
18|acc:0.9612
19|acc:0.9549
20|acc:0.9549
21|acc:0.9599
22|acc:0.9562
23|acc:0.9562
24|acc:0.9574
25|acc:0.9562
26|acc:0.9549
27|acc:0.9574
28|acc:0.9549
29|acc:0.9562


In [14]:
run(
    "3d_recognition.py",
    "--epochs 20 --cnn C-32-3-1-same,C-32-3-1-same,M-3-2,C-64-3-1-same,C-64-3-1-same,M-3-2,F"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://bd50c7c5.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.8425
1|acc:0.9175
2|acc:0.9400
3|acc:0.9575
4|acc:0.9500
5|acc:0.9525
6|acc:0.9625
7|acc:0.9650
8|acc:0.9475
9|acc:0.9650
10|acc:0.9600
11|acc:0.9625
12|acc:0.9625
13|acc:0.9600
14|acc:0.9625
15|acc:0.9625
16|acc:0.9600
17|acc:0.9675
18|acc:0.9650
19|acc:0.9625


In [15]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 30 --cnn C-64-3-1-same,C-64-3-1-same,M-3-2,C-128-3-1-same,C-128-3-1-same,M-3-2,C-128-3-1-same,F --modelnet_dim 32 --train_split 0.8 --batch_evaluation True"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= https://bd50c7c5.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.2328
1|acc:0.2328
2|acc:0.2328
3|acc:0.2328
4|acc:0.2328
5|acc:0.2328


KeyboardInterrupt: ignored

In [16]:
run(
    "3d_recognition.py",
    "--epochs 20 --cnn C-32-3-1-same,C-32-3-1-same,M-3-2,C-64-3-1-same,C-64-3-1-same,M-3-2,F --batch_evaluation True"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=False
tensorboard url= http://4e4b93d2.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.8425
1|acc:0.9150
2|acc:0.9375
3|acc:0.9575
4|acc:0.9500
5|acc:0.9575
6|acc:0.9625
7|acc:0.9625
8|acc:0.9425
9|acc:0.9625
10|acc:0.9550
11|acc:0.9625
12|acc:0.9625
13|acc:0.9650
14|acc:0.9600
15|acc:0.9650
16|acc:0.9575
17|acc:0.9625
18|acc:0.9675
19|acc:0.9675


In [6]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 30 --cnn C-128-5-2-same,C-64-3-1-same,C-64-3-1-same,M-3-2,C-128-3-1-same,C-128-3-1-same,M-3-2,F,R-128,D --modelnet_dim 32 --train_split 0.9 --batch_evaluation True"
)

<<= <=> =>>
calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ...
calling unzip ngrok-stable-linux-amd64.zip ...
ngrok installed. path=/content/ngrok
status: tensorboard=False, ngrok=False
tensorboard url= http://7733a42e.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.6350
1|acc:0.8175
2|acc:0.9000
3|acc:0.9125
4|acc:0.8950
5|acc:0.9325
6|acc:0.9425
7|acc:0.9575
8|acc:0.9500
9|acc:0.9525
10|acc:0.9525
11|acc:0.9625
12|acc:0.9675
13|acc:0.9550
14|acc:0.9600
15|acc:0.9575
16|acc:0.9650
17|acc:0.9625
18|acc:0.9550
19|acc:0.9550
20|acc:0.9600
21|acc:0.9600
22|acc:0.9600
23|acc:0.9625
24|acc:0.9600
25|acc:0.9575
26|acc:0.9600
27|acc:0.9575
28|acc:0.9575
29|acc:0.9575


In [7]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 30 --cnn C-64-5-3-same,C-32-3-1-same,C-32-3-1-same,M-3-2,C-64-3-1-same,C-64-3-1-same,M-3-2,F,R-128,D --modelnet_dim 32 --train_split 0.9 --batch_evaluation True"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://7733a42e.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.7275
1|acc:0.8625
2|acc:0.9175
3|acc:0.9325
4|acc:0.9300
5|acc:0.9375
6|acc:0.9475
7|acc:0.9450
8|acc:0.9425
9|acc:0.9525
10|acc:0.9600
11|acc:0.9525
12|acc:0.9500
13|acc:0.9550
14|acc:0.9550
15|acc:0.9500
16|acc:0.9575
17|acc:0.9550
18|acc:0.9575
19|acc:0.9500
20|acc:0.9600
21|acc:0.9500
22|acc:0.9550
23|acc:0.9550
24|acc:0.9500
25|acc:0.9550
26|acc:0.9575
27|acc:0.9550
28|acc:0.9575
29|acc:0.9550


In [8]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 30 --cnn C-64-5-3-same,C-32-3-1-same,M-3-2,C-64-3-1-same,C-64-3-1-same,M-3-2,F,R-128,D --modelnet_dim 32 --train_split 0.9 --batch_evaluation True"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://7733a42e.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.7625
1|acc:0.8650
2|acc:0.9250
3|acc:0.9250
4|acc:0.9525
5|acc:0.9600
6|acc:0.9575
7|acc:0.9575
8|acc:0.9600
9|acc:0.9625
10|acc:0.9625
11|acc:0.9625
12|acc:0.9625
13|acc:0.9650
14|acc:0.9650
15|acc:0.9600
16|acc:0.9700
17|acc:0.9650
18|acc:0.9675
19|acc:0.9675
20|acc:0.9650
21|acc:0.9625
22|acc:0.9600
23|acc:0.9600
24|acc:0.9625
25|acc:0.9625
26|acc:0.9650
27|acc:0.9575
28|acc:0.9650
29|acc:0.9675


In [9]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 30 --cnn C-64-5-3-same,C-32-3-1-same,M-3-2,C-32-3-1-same,C-32-3-1-same,M-3-2,F,R-32,D --modelnet_dim 32 --train_split 0.9 --batch_evaluation True"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://7733a42e.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.7600
1|acc:0.8150
2|acc:0.8725
3|acc:0.8875
4|acc:0.9175
5|acc:0.9225
6|acc:0.9225
7|acc:0.9300
8|acc:0.9075
9|acc:0.9250
10|acc:0.9425
11|acc:0.9450
12|acc:0.9475
13|acc:0.9425
14|acc:0.9425
15|acc:0.9575
16|acc:0.9525
17|acc:0.9425
18|acc:0.9550
19|acc:0.9500
20|acc:0.9475
21|acc:0.9550
22|acc:0.9575
23|acc:0.9575
24|acc:0.9550
25|acc:0.9550
26|acc:0.9600
27|acc:0.9550
28|acc:0.9525
29|acc:0.9600


In [10]:
run(
    "3d_recognition.py",
    "--batch_size 256 --epochs 30 --cnn C-64-5-3-same,C-32-3-1-same,M-3-2,C-64-3-1-same,C-64-3-1-same,M-3-2,F,R-64,D --modelnet_dim 32 --train_split 0.9 --batch_evaluation True"
)

Clearing cache...done
<<= <=> =>>
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= https://7733a42e.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.5125
1|acc:0.7150
2|acc:0.8400
3|acc:0.8875
4|acc:0.8875
5|acc:0.8900
6|acc:0.9200
7|acc:0.9275
8|acc:0.9325
9|acc:0.9275
10|acc:0.9350
11|acc:0.9400
12|acc:0.9400
13|acc:0.9525
14|acc:0.9475
15|acc:0.9500
16|acc:0.9500
17|acc:0.9525
18|acc:0.9525
19|acc:0.9500
20|acc:0.9475
21|acc:0.9525
22|acc:0.9525
23|acc:0.9575
24|acc:0.9525
25|acc:0.9500
26|acc:0.9500
27|acc:0.9575
28|acc:0.9600
29|acc:0.9575


In [18]:
run(
    "3d_recognition.py",
    "--batch_size 128 --epochs 30 --cnn C-64-5-3-same,C-32-3-1-same,M-3-2,C-64-3-1-same,C-64-3-1-same,M-3-2,F,R-128,D --modelnet_dim 32 --train_split 0.9 --batch_evaluation True"
)

<<= <=> =>>
calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ...
calling unzip ngrok-stable-linux-amd64.zip ...
ngrok installed. path=/content/drive/DL_3drecognition/ngrok
status: tensorboard=True, ngrok=True
tensorboard url= http://ff1508a7.ngrok.io
<<= <=> =>>
[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.7900
1|acc:0.8900
2|acc:0.9250
3|acc:0.8975
4|acc:0.9500
5|acc:0.9575
6|acc:0.9475
7|acc:0.9650
8|acc:0.9575
9|acc:0.9625
10|acc:0.9625
11|acc:0.9600
12|acc:0.9675
13|acc:0.9625
14|acc:0.9675
15|acc:0.9625
16|acc:0.9700
17|acc:0.9625
18|acc:0.9625
19|acc:0.9625
20|acc:0.9625
21|acc:0.9625
22|acc:0.9650
23|acc:0.9650
24|acc:0.9650
25|acc:0.9625
26|acc:0.9625
27|acc:0.9650
28|acc:0.9650
29|acc:0.9625
